In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the Oxford-IIIT Pet dataset
(ds_train, ds_test), ds_info = tfds.load('caltech101', split=['train', 'test'], as_supervised=True, with_info=True)

# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (32, 32))
    return image, label

# Apply resizing to the dataset
ds_train = ds_train.map(resize_image)
ds_test = ds_test.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the datasets
ds_train = ds_train.shuffle(10000).batch(64)
ds_test = ds_test.batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=101)  
ds_train = ds_train.map(categorical_labels)
ds_test = ds_test.map(categorical_labels)

# Verify the shapes
for images, labels in ds_train.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

for images, labels in ds_test.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\sadia_tisha1\tensorflow_datasets\caltech101\3.0.1.incompleteI3D7OE\caltech101-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\sadia_tisha1\tensorflow_datasets\caltech101\3.0.1.incompleteI3D7OE\caltech101-test.tfrecord…

Dataset caltech101 downloaded and prepared to C:\Users\sadia_tisha1\tensorflow_datasets\caltech101\3.0.1. Subsequent calls will reuse this data.
Images shape: (64, 32, 32, 3)
Labels shape: (64, 101)
Images shape: (64, 32, 32, 3)
Labels shape: (64, 101)


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(101, activation='softmax'))  # Change the number of units and activation
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(ds_train, batch_size=64, epochs=50)

Epoch 1/50
48/48 [==============================] - 167s 3s/step - loss: 4.9561 - accuracy: 0.0879
Epoch 2/50
48/48 [==============================] - 215s 4s/step - loss: 3.7307 - accuracy: 0.2249
Epoch 3/50
48/48 [==============================] - 204s 4s/step - loss: 3.3236 - accuracy: 0.2945
Epoch 4/50
48/48 [==============================] - 204s 4s/step - loss: 2.9345 - accuracy: 0.3857
Epoch 5/50
48/48 [==============================] - 206s 4s/step - loss: 2.3783 - accuracy: 0.5021
Epoch 6/50
48/48 [==============================] - 203s 4s/step - loss: 2.1349 - accuracy: 0.5793
Epoch 7/50
48/48 [==============================] - 202s 4s/step - loss: 1.9424 - accuracy: 0.6646
Epoch 8/50
48/48 [==============================] - 199s 4s/step - loss: 1.6507 - accuracy: 0.6943
Epoch 9/50
48/48 [==============================] - 204s 4s/step - loss: 2.2859 - accuracy: 0.5999
Epoch 10/50
48/48 [==============================] - 205s 4s/step - loss: 2.7569 - accuracy: 0.5361
Epoch 11/

In [4]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


Test Accuracy: 0.13853738701725554
Accuracy: 0.13853738701725554
F1 Score: 0.15907598646576115
